# Bayes ingenuo

In [78]:
from __future__ import division
import numpy as np
from collections import defaultdict, Counter
from operator import itemgetter

S = [('Amo esa cosa','positivo'),('Me gusta el chocolate','positivo'),('Odio esa cosa','negativo'),('Amo mi trabajo','positivo'),('Detesto tu camisa','negativo')]
   
print S


[('Amo esa cosa', 'positivo'), ('Me gusta el chocolate', 'positivo'), ('Odio esa cosa', 'negativo'), ('Amo mi trabajo', 'positivo'), ('Detesto tu camisa', 'negativo')]


In [79]:
def vocab():
    dicc = defaultdict()
    dicc.default_factory = lambda: len(dicc)
    return dicc

def BoW(corpus,vocab):
    for w in corpus:
        yield[vocab[w_i] for w_i in w.lower().split()]
        
words = vocab()
frases = list(BoW( [s[0] for s in S],words ))

print frases

[[0, 1, 2], [3, 4, 5, 6], [7, 1, 2], [0, 8, 9], [10, 11, 12]]


In [80]:
frec_clase = Counter([s[1] for s in S])

print frec_clase

Counter({'positivo': 3, 'negativo': 2})


In [ ]:

def p(f_conj,f_cond,l=0.01):
    return (f_conj+l)/(f_cond+l*len(words))

P_c = np.zeros(len(frec_clase))
for i,(c,f) in enumerate(frec_clase.iteritems()):
    P_c[i] = p(f,len(S))


In [81]:
print P_c[0]
print P_c[1]

0.586744639376
0.391812865497


In [82]:
train_corpus = defaultdict(list)
for s in S:
    for word in s[0].lower().split():
        train_corpus[s[1]].append(word)

In [83]:
print train_corpus['positivo']

['amo', 'esa', 'cosa', 'me', 'gusta', 'el', 'chocolate', 'amo', 'mi', 'trabajo']


In [84]:
A = np.zeros((len(words),len(train_corpus)))

for i,(clase,w) in enumerate(train_corpus.iteritems()):
    frecs = Counter(w)
    for word in words:
        if word in w:
            A[words[word],i] =  p(frecs[word],frec_clase[clase])
        else:
            A[words[word],i] = p(0,frec_clase[clase])

In [87]:
print 'Probabilidad de clases: ',P_c
print 'Matriz de probabilidades: ','\n', A

Probabilidad de clases:  [ 0.58674464  0.39181287]
Matriz de probabilidades:  
[[ 0.64217252  0.00469484]
 [ 0.32268371  0.4741784 ]
 [ 0.32268371  0.4741784 ]
 [ 0.32268371  0.00469484]
 [ 0.32268371  0.00469484]
 [ 0.32268371  0.00469484]
 [ 0.32268371  0.00469484]
 [ 0.00319489  0.4741784 ]
 [ 0.32268371  0.00469484]
 [ 0.32268371  0.00469484]
 [ 0.00319489  0.4741784 ]
 [ 0.00319489  0.4741784 ]
 [ 0.00319489  0.4741784 ]]


# Proceso de evaluación

In [88]:
def prob_clase(sent,clase_id):
    prob = 1.0
    for word in sent.lower().split():
        try:
            prob *= A[words[word],clase_id]
        except:
            prob *= 1.0
    return prob

In [90]:
eval_sent = 'odio la vida'
print prob_clase(eval_sent,1)

0.474178403756


In [92]:
def predict(sent):
    clases = {}
    for i,c in enumerate(frec_clase): 
        clases[c] = prob_clase(sent,i)
    return max(clases.iteritems(), key=itemgetter(1))[0]

In [93]:
print predict(eval_sent)

negativo


In [94]:
print predict('Amo la forma en que me miras')

positivo
